In [ ]:
# Code is only for demonstration purpose and not executable in Jupyter Notebook.
# Complete, executable script can be found in the deliverables

import numpy as np
import cv2 as cv

def extractKeypoints(img1, img2, N):
    sift = cv.SIFT_create()
    keypoint1, descriptor1 = sift.detectAndCompute(img1, None)
    keypoint2, descriptor2 = sift.detectAndCompute(img2, None)

    if keypoint1 is None or keypoint2 is None or len(keypoint1) < N or len(keypoint2) < N:
        return np.array([]), np.array([])

    bf = cv.BFMatcher_create()
    matches = bf.knnMatch(descriptor1, descriptor2, k = 2)

    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)

    matches = sorted(good, key = lambda x:x.distance)[:N]

    points1 = np.float32([keypoint1[m.queryIdx].pt for m in matches])
    points2 = np.float32([keypoint2[m.trainIdx].pt for m in matches])

    return points1, points2

[pts1, pts2] = ExtractKeypoints.extractKeypoints(imgOriginal, imgCropped, 25)
[H, status] = cv.findHomography(pts2, pts1, method = cv.RANSAC)

In [ ]:
# Code is only for demonstration purpose and not executable in Jupyter Notebook.
# Complete, executable script can be found in the deliverables

angle = abs(math.atan2(H[1,0], H[0,0])) * 180 / math.pi
if angle < 5:
    originalShape = imgOriginal.shape
    croppedShape = imgCropped.shape
    croppedPts = np.array([[0, 0, 1], [0, croppedShape[1], 1], [croppedShape[0], croppedShape[1], 1], [croppedShape[0], 0, 1]])
    originalPts = np.matmul(H, croppedPts.transpose()).transpose()
    originalPts = np.round(originalPts[:, 0:2] / originalPts[:, 2:3]).astype(np.int32)
    x1 = np.min(originalPts[:, 0]).clip(0, originalShape[1])
    x2 = np.max(originalPts[:, 0]).clip(0, originalShape[1])
    y1 = np.min(originalPts[:, 1]).clip(0, originalShape[0])
    y2 = np.max(originalPts[:, 1]).clip(0, originalShape[0])

In [ ]:
# Code is only for demonstration purpose and not executable in Jupyter Notebook.
# Complete, executable script can be found in the deliverables

class PreprocessImage(object):

    @classmethod
    def preprocess(cls, img, normalize):
        shape = img.shape
        intensity = (0.11 * img[:,:,0] + 0.59 * img[:,:,1] + 0.30 * img[:,:,2])
        intensity = intensity.reshape((shape[0], shape[1], 1))
        edge = cv.Canny(img, 100, 100, apertureSize = 3)
        # cv.imshow('img', edge)
        # cv.waitKey(0)
        edge = edge
        edge = edge.reshape((shape[0], shape[1], 1))
        if normalize is True:
            intensity = intensity / 255.0
            edge = edge / 255.0
            img = img / 255.0
            result = np.append(img, intensity, axis = -1)
            result = np.append(result, edge, axis = -1)
            result = np.moveaxis(result, -1, 0)
            result[[0,2]] = result[[2,0]]
            return result.astype(float)
        else:
            result = np.append(img, intensity, axis = -1)
            result = np.append(result, edge, axis = -1)
            result = np.moveaxis(result, -1, 0)
            result[[0,2]] = result[[2,0]]
            return result.astype(np.uint8)